# Final 2022 Spring

###### Created by Qihang Ma -- 2023.04.27

In [1]:
import warnings
warnings.filterwarnings("ignore")
from RiskLib import calculation, cov_matrix, linear_regression, optimal_portfolio, risk_parity, risk_attribution, Option, simulation, VaR
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import fsolve, minimize
import statsmodels.api as sm
from scipy.stats import t, norm, kurtosis, skew
import matplotlib.pyplot as plt


## Problem 1

Why do we need risk management?

1. Minimize Losses: Risk management allows us to identify potential risks and take proactive measures to minimize losses in the event of an incident or unexpected occurrence.

2. Compliance: Various industries have regulatory and compliance requirements that necessitate risk management to ensure legal compliance and prevent potential penalties.

3. Protects reputation: Effective risk management can help organizations safeguard their reputation by identifying and mitigating risks before they become crises.

4. Business Continuity: Risk management ensures that organizations can continue to operate during a crisis, reducing the impact of negative events on the organization's operations and stakeholders.

5. Prioritization: Risk management allows organizations to prioritize resources and make informed decisions by considering potential risks and rewards.

6. Improved Decision Making: Risk management encourages organizations to gather and analyze data to inform their decision-making process, leading to better and more informed decisions.

- In summary, risk management is essential to identify and manage potential risks and reduce their negative impact on organizations, individuals, and society.

## Problem 2

Use data in question2.csv.

- a. What are the first 4 moments of this data?

- b. If this represented the Profit and Loss from an investment, what would you say about its risk and reward?

In [2]:
pnl = pd.read_csv('question2.csv')

In [3]:
print("Mean: {:.4f}.".format(np.mean(pnl.values)))
print("Standard Deviation: {:.4f}.".format(np.std(pnl.values)))
print("Variance: {:.4f}.".format(np.var(pnl.values)))
print("Skewness: {:.4f}.".format(skew(pnl)[0]))
print("kurtosis: {:.4f}.\n".format(kurtosis(pnl)[0]))


print("VaR: {:.4f}".format(VaR.calculate_var(pnl)))
print("ES: {:.4f}".format(VaR.calculate_ES(pnl.values.T[0])))


Mean: 0.6722.
Standard Deviation: 10.5493.
Variance: 111.2868.
Skewness: -1.6289.
kurtosis: 2.2282.

VaR: 21.9104
ES: 28.8098


Negative Skew and excess kurtosis make this a risky investment even if it has a positive expected return.

This is seen with the large VaR and ES numbers.

## Problem 3

Use data in question3.csv.

- a. This data has missing values. Calculate the pairwise correlation matrix.

- b. Is this matrix PSD?
    
- c. If the matrix is not PSD, use Higham 2002 to calculate the nearest PSD matrix.

In [4]:
missing_data = pd.read_csv('question3.csv')
pairwise_corr = cov_matrix.missing_cov(missing_data.values, False)
pd.DataFrame(pairwise_corr)

,0,1,2
0,1.00000,-0.620160,0.829550
1,-0.62016,1.000000,-0.989449
2,0.82955,-0.989449,1.000000


In [5]:
cov_matrix.is_psd(pairwise_corr)

False

In [6]:
Fixed_Matrix = pd.DataFrame(cov_matrix.Higham_psd(pairwise_corr))
Fixed_Matrix

,0,1,2
0,1.000000,-0.629592,0.816847
1,-0.629592,1.000000,-0.962454
2,0.816847,-0.962454,1.000000


In [7]:
cov_matrix.is_psd(Fixed_Matrix)

True

## Problem 4 

Assume you have 3 variables

- 𝑌1, 𝑌2, 𝑋

Each are assumed to be distributed normally. Further assume you have a structural model: 

- 𝑌1 = α1 + β1𝑋 + ε1

- 𝑌2 = α2+β2𝑋 +ε2

And ε1&ε2 are assumed to be independent and normally distributed. Describe how you would jointly simulate 𝑌1, 𝑌2, & 𝑋.

- Calculate the covariance of X, Y1, and Y2.  Check that the matrix is PSD.  Simulate from that covariance matrix.

- No need to fit the structural model because everything is normally distributed and the conditional distributions are the same 

## Problem 5

Assume the have the same three variables and structural model as in Question #4. However, now we assume that ε1 & ε2 are independent and distributed by a generalized Student T distribution.

Describe how you would jointly simulate 𝑌1, 𝑌2, & 𝑋.

- fit the models to get the parameters and U values for the e1 and e2 variables.  Find the U values for x

- Use the spearman correlation between the e1, e2, and X U values to fit the Gaussian copula.

- Simulate from the copula and transform back to e1, e2, and x

- use the fitted Alpha and Beta values to transform e1, e2, and X into Y1 and Y2.

## Problem 6

Assume

- European Put, value = 7.18
- Risk Free Rate = 2%, no dividend payment
- Stock Price $100, Strike Price $100
- 1 Year to maturity

a. What is the implied volatility?

b. Assume the stock returns are normally distributed, Expected annual return = 4%, with an annual volatility equal to the implied volatility, and 255 trading days in a year. The implied volatility does not change. A market maker has sold this option. What is her 1-day VaR and ES expressed in $ terms?

In [8]:
value = 7.18
rf = 0.02
q = 0
S0 = 100
K = 100
T = 1

ivol = Option.implied_volatility_bs(S0, K, T, rf, q, value, 'put')
print("Implied Volatility: {:.4f}" .format(ivol))

Implied Volatility: 0.2062


In [9]:
T1 = 364/365
VaRs = []
ESs = []

for i in range(1000,2000):

    np.random.seed(i)
    sim_r = np.random.normal(size=10000, loc=0.04/255, scale=ivol/np.sqrt(255))
    sim_price = pd.DataFrame((1+sim_r) * 100, columns=['Price'])

    sim_price['PNL'] = sim_price.apply(lambda x: Option.black_scholes(x,K,T1,rf,q,ivol,'put')-value)

    VaRs.append(VaR.calculate_var(sim_price['PNL']))
    ESs.append(VaR.calculate_ES(sim_price['PNL']))

VaRs = np.array(VaRs)
ESs = np.array(ESs)

print("VaR Mean: {:.4f} -- 5% range [{:.4f}, {:.4f}].".format(VaRs.mean(), np.quantile(VaRs, 0.025), np.quantile(VaRs, 0.975)))
print("ES Mean: {:.4f} -- 5% range [{:.4f}, {:.4f}].".format(ESs.mean(), np.quantile(ESs, 0.025), np.quantile(ESs, 0.975)))


VaR Mean: 0.8659 -- 5% range [0.8458, 0.8866].
ES Mean: 1.0666 -- 5% range [1.0452, 1.0897].


## Problem 7

The market maker from question #6 wishes to hedge her position by buying or selling stock. How many shares should she buy/sell for each option she has sold?

In [10]:
delta = Option.black_scholes_matrix(S0,K,T,rf,q,ivol,'put').delta() * (-1)

print('She should sell {:.2f} shares for each share she sold.'.format(delta))

She should sell 0.42 shares for each share she sold.


## Problem 8

Given 3 Assets, A, B, and C. The risk free rate is 0%. 

The correlation matrix is:3×3 Matrix{Float64}:

|    | A  | B  | C  |
| ---| ---| ---| ---|
| A  | 1.0| 0.5| 0.5|
| B  | 0.5| 1.0| 0.5|
| C  | 0.5| 0.5| 1.0|


Volatilities are

| A  | B  | C  |
| ---| ---| ---|
| 0.1| 0.2| 0.3|


Expected Returns are

| A  | B  | C  |
| ---| ---| ---|
| 0.03| 0.06| 0.09|


What is the maximum sharpe ratio portfolio with no constraints on negative weights?

In [11]:
assets = ['A','B','C']
rf = 0
vol = np.array([0.1,0.2,0.3])
exp_return = np.array([0.03,0.06,0.09])
corr_matrix = np.full((3,3),0.5)

for i in range(3):
    for j in range(3):
        if i == j:
            corr_matrix[i,j]=1

covariance_matrix = pd.DataFrame(np.diag(vol)@corr_matrix@np.diag(vol),index=assets, columns=assets)

In [12]:
weights, sharp_ratio = optimal_portfolio.Optweight_sr(assets, exp_return,covariance_matrix, rf, (None,None))
weights

,Stock,Weight,cEr
0,A,0.545261,0.016358
1,B,0.272899,0.016374
2,C,0.181840,0.016366


## Problem 9 

Given the covariance structure in #8,

- a. if we have an equal weight on each asset (33.3...%), what is the risk contribution of each asset?

- b. What are the risk parity portfolio weights?

In [13]:
weights = np.array([1/3]*3)
risk_attribution.risk_contrib(weights,covariance_matrix)

A    0.023333
B    0.053333
C    0.090000
dtype: float64

In [14]:
risk_parity.vol_risk_parity(exp_return,covariance_matrix)

,Weight,cEr,CSD
A,0.545454,0.016364,0.044536
B,0.272727,0.016364,0.044536
C,0.181818,0.016364,0.044536


## Problem 10

Use Data in question10.csv for assets A, B, & C, and given a starting weight of [0.55, 0.27, 0.18]

 Calculate the ex-post return contribution of each asset

In [15]:
returns = pd.read_csv('question10.csv')
start_weights = np.array([0.55, 0.27, 0.18])

risk_attribution.expost_attribution(start_weights, returns)

,Value,A,B,C,Portfolio
0,TotalReturn,0.053001,-0.089135,0.182844,0.037996
1,Return Attribution,0.030156,-0.027476,0.035316,0.037996
2,Vol Attribution,0.013933,0.015537,0.013961,0.043431


## EC 1

Using the data in ec1.csv for assets A and B. This is output from a simulation of asset values.

- Calculate the covariance of the assets. Use the expected value from the series. Assume the risk free rate is 2%. Weights must be >=-1.0. Find the maximum sharpe ratio portfolio.
(HINT: Because you have 2 Assets, you can quickly sweep the potential weights to find maximum values instead of using an optimizer)

- Using the simulated values in the CSV file, calculate the portfolio that maximizes
(μ−𝑟𝑓/𝐸𝑆) Where 𝐸𝑆 is the portfolio expected shortfall.

-  Why are these two portfolios so different?


In [16]:
dataset = pd.read_csv('ec1.csv')
assets = ['A', 'B']

#### Optimize with sharp ratio

In [17]:
covar = dataset.cov()
exp = dataset.mean()
weights_sr, sharp_ratio = optimal_portfolio.Optweight_sr_two_assets(assets, exp, covar, 0.02, (-1,2))
weights_sr

,Weight
A,2.0
B,-1.0


#### Optimize with ES

In [18]:
def Optweight_esr(stockMeans, stockreturn, rf, bound=(0,None)):

    # Define Sharpe ratio function
    def esr(w):
        m = np.dot(w, stockMeans) - rf
        s = VaR.calculate_ES(np.dot(stockreturn,w))
        return m / s

    n = stockMeans.shape[0]

    # Define optimization problem
    bounds = [bound] * n
    cons = {"type": "eq", "fun": lambda w: np.sum(w) - 1}
    result = minimize(lambda w: -esr(w), np.ones(n) / n, method="SLSQP", bounds=bounds, constraints=cons)

    # Extract optimal weights and other information
    w = result.x
    w = w / np.sum(w)
    OptWeights = pd.DataFrame({"Weight": w, "cEr": stockMeans * w})

    return OptWeights

In [19]:
weights_esr = Optweight_esr(exp, dataset, 0.02, (-1, None))
weights_esr

,Weight,cEr
A,0.928351,0.056702
B,0.071649,-0.026205


#### Compare two portfolio

In [20]:
portfolio_return_sr = np.dot(dataset, weights_sr['Weight'].values)
portfolio_return_esr = np.dot(dataset, weights_esr['Weight'].values)

VaR_sr = VaR.calculate_var(portfolio_return_sr)
VaR_esr = VaR.calculate_var(portfolio_return_esr)

ES_sr = VaR.calculate_ES(portfolio_return_sr)
ES_esr = VaR.calculate_ES(portfolio_return_esr)


print("For first portfolio, VaR: {:.4f}, ES: {:.4f}".format(VaR_sr, ES_sr))
print("For Second portfolio, VaR: {:.4f}, ES: {:.4f}\n".format(VaR_esr, ES_esr))

For first portfolio, VaR: 2.9507, ES: 4.0576
For Second portfolio, VaR: 0.0717, ES: 0.0717



In [21]:
print("Mean: {:.4f}.".format(np.mean(dataset['A'].values)))
print("Standard Deviation: {:.4f}.".format(np.std(dataset['A'].values)))
print("Variance: {:.4f}.".format(np.var(dataset['A'].values)))
print("Skewness: {:.4f}.".format(skew(dataset['A'])))
print("kurtosis: {:.4f}.".format(kurtosis(dataset['A'])))

Mean: 0.0611.
Standard Deviation: 0.1879.
Variance: 0.0353.
Skewness: -0.1278.
kurtosis: -0.1427.


In [22]:
print("Mean: {:.4f}.".format(np.mean(dataset['B'].values)))
print("Standard Deviation: {:.4f}.".format(np.std(dataset['B'].values)))
print("Variance: {:.4f}.".format(np.var(dataset['B'].values)))
print("Skewness: {:.4f}.".format(skew(dataset['B'])))
print("kurtosis: {:.4f}.".format(kurtosis(dataset['B'])))

Mean: -0.3657.
Standard Deviation: 1.1809.
Variance: 1.3945.
Skewness: 2.2284.
kurtosis: 4.9241.


## EC 2

Using the data in ec2.csv.
- a. 𝑋~𝑁(μ, σ)
- b. 𝑌𝑖 =α𝑖 +β𝑖𝑋 +ε𝑖 𝑓𝑜𝑟𝑖 ∈ [1,2]
- c. ε𝑖~𝑇(0,σ𝑖,ν𝑖)𝑓𝑜𝑟𝑖 ∈ [1,2]
- d. Initial price of Y1 = $10.
- e. Initial price of Y2 = $50.
- f. You hold a portfolio of 100 shares of both Y1 and Y2

What is the VaR and ES of the portfolio, given the information above, expressed as a $ profit and loss?

In [23]:
dataset = pd.read_csv('ec2.csv')

In [24]:
parameters_y1, error_y1 = linear_regression.fit_regression_t(dataset['y1'],dataset['x'])
parameters_y2, error_y2 = linear_regression.fit_regression_t(dataset['y2'],dataset['x'])

In [25]:
VaRs = []
ESs = []

for i in range(100):
    fitting_data  = pd.DataFrame({'error_y1': error_y1,
                                'error_y2': error_y2,
                                'X': dataset['x']})
    fitting_model = np.array(['t','t','n'])
    sim_return, p = simulation.gaussian_copula(fitting_data,fitting_model,seed=i)
    sim_return['y1'] = sim_return.apply(lambda x: x[2]*parameters_y1['beta']+parameters_y1['alpha']+x[0], axis=1)
    sim_return['y2'] = sim_return.apply(lambda x: x[2]*parameters_y2['beta']+parameters_y2['alpha']+x[1], axis=1)
    pnl = sim_return[['y1', 'y2']] @ np.array([10,50]) * 100
    VaRs.append(VaR.calculate_var(pnl))
    ESs.append(VaR.calculate_ES(pnl))
VaRs = np.array(VaRs)
ESs = np.array(ESs)


In [26]:
print("VaR Mean: {:.4f} -- 5% range [{:.4f}, {:.4f}].".format(VaRs.mean(), np.quantile(VaRs, 0.025), np.quantile(VaRs, 0.975)))
print("ES Mean: {:.4f} -- 5% range [{:.4f}, {:.4f}].".format(ESs.mean(), np.quantile(ESs, 0.025), np.quantile(ESs, 0.975)))

VaR Mean: 159.6873 -- 5% range [155.4005, 164.3533].
ES Mean: 203.2695 -- 5% range [197.8820, 207.9210].
